In [1]:
import pandas as pd
import re
import requests
from nltk.util import bigrams

import time
from datetime import datetime 
import pickle

start_time = datetime.now() 

In [2]:
# upload sentences with summarizing feature
bef = pd.read_csv('sentences_final_after_check.csv')
bef

,sentence,no_change,summarizing
0,והנפיחות בעיניים?,1,0
1,"ככלל, כדאי להיות קשובים לגוף וכשאשר מופיעים סי...",0,1
2,הצגה עצמיתתחילה תתבקש לפרט את המחלות או הליקוי...,0,1
3,מגמת משרד החינוך הינה לשלב את הילד בעל הצרכים ...,0,1
4,"בייעוץ כזה יובאו בחשבון המסלול, משך הטיול, אופ...",0,1
...,...,...,...
516,"הורים יקרים,במדריך הזה מפורט התהליך העומד בפני...",0,1
517,מהיום ניתן לקבל את חשבון הארנונה ישירות לדואר...,0,1
518,".תורם לאיכות הסביבה, קבלת חשבון ארנונה תקופתי ...",0,1
519,ניתן לקבל את חשבון הארנונה למייל במילוי טופס.,0,1


In [ ]:
# make request to ONLP site API (should be done after all instructions on their site and connecting)
# and download dependency tree for each sentence
dep_tree_list = []
for s in bef.sentence:
    no_q = re.sub("\"", "", s)
    text = no_q

    localhost_yap = "http://localhost:8000/yap/heb/joint"
    data = '{{"text": "{}  "}}'.format(text).encode('utf-8')  # input string ends with two space characters
    headers = {'content-type': 'application/json'}
    response = requests.get(url=localhost_yap, data=data, headers=headers)
    json_response = response.json()

    heads_tree = []
    heads = []
    dep_tree = json_response['dep_tree']
    dep_tree2 = dep_tree.split()
    # save dependency tree as string into list
    dep_tree_list.append(dep_tree)
    # we need to sleep the procces for some time to prevent killing process if not enough CPU, RAM etc
    time.sleep(20)
    
print('Time elapsed in (hh:mm:ss.ms): "{}"'.format(datetime.now() - start_time))

In [ ]:
# Save a dictionary into a pickle file.
import pickle

for i in range(len(dep_tree_list)):
    pickle.dump(dep_tree_list[i], open( "pickle_sentences/sentence" + str(i+1) + ".p", "wb" ) )
    
# Load the dictionary back from the pickle file.
#import pickle
#dep_tree_list0 = pickle.load( open( "pickle_sentences/sentence3.p", "rb" ) )
#print(dep_tree_list0)

In [6]:
# creating main features

words = []
letters = []
word_avg = []
numbers = []
symbols = []
if_question = []

punctuation = []
sym_hashtag = []
sym_shtrudel = []
sym_dollar = []
sym_ampersand = []
sym_percend = []
sym_star = []
sym_tilda = []
sym_math = []
sym_logic = []
sym_brackets = []
sym_num = []
sym_quotes = []
sym_slashes = []

bigrams = []

for s in bef.sentence:
    
    bigrams.append(len(s.split())-1)
    
    val = re.findall(r'\w+', s)
    words.append(len(val))
    
    val = re.findall(r'\w', s)
    letters.append(len(val))
    
    val = re.findall(r'\w+', s)
    average = sum(len(word) for word in val) / len(val)
    word_avg.append(round(average, 3))
    
    val = re.findall(r'[0-9]', s)
    numbers.append(len(val))
    
    val = re.findall(r'[^א-ת0-9 ]', s)
    symbols.append(len(val))
    
    val = re.findall(r'[`!^:;\',.?]', s)
    punctuation.append(len(val))
    
    val = re.findall(r'[#]', s)
    sym_hashtag.append(len(val))
    
    val = re.findall(r'[@]', s)
    sym_shtrudel.append(len(val))
    
    val = re.findall(r'[$]', s)
    sym_dollar.append(len(val))
    
    val = re.findall(r'[&]', s)
    sym_ampersand.append(len(val))
    
    val = re.findall(r'[%]', s)
    sym_percend.append(len(val))
    
    val = re.findall(r'[*]', s)
    sym_star.append(len(val))
    
    val = re.findall(r'[~]', s)
    sym_tilda.append(len(val))
    
    val = re.findall(r'[\+\-\*\/<>=%^]', s)
    sym_math.append(len(val))
    
    val = re.findall(r'[&|]', s)
    sym_logic.append(len(val))
    
    val = re.findall(r'[()[]{}]', s)
    sym_brackets.append(len(val))
    
    val = re.findall(r'[№]', s)
    sym_num.append(len(val))
    
    val = re.findall(r'["\']', s)
    sym_quotes.append(len(val))
    
    val = re.findall(r'[\//]', s)
    sym_slashes.append(len(val))
    
    if s[len(s)-1]=='?':
        if_question.append(1)
    else:
        if_question.append(0)


In [ ]:
# creating POS features (Part Of Speach features)
# tree depth and MDD(Mean Dependency Distance)

NN_l = []
VB_l = []
JJ_l = []
PREPOSITION_l = []
DEF_l = []
CONJ_l = []
BN_l = []
NNP_l = []
NNT_l = []
TEMP_l = []
RB_l = []
CC_l = []
AT_l = []
QW_l = []
REL_l = []
PRP_l = []
IN_l = []
DTT_l = []
DT_l = []
MD_l = []

t_depth = []
mdd = []


for s in range(len(dep_tree_list)):
    heads_tree = []
    heads = []
    dep_tree2 = dep_tree_list[s]
    # check for each sentence(dep_tree) where it ends and add options to another list
    for i in range(len(dep_tree2)):
        if i<len(dep_tree2)-1:
            if dep_tree2[7]=='_' and dep_tree2[8]=='_':
                heads_tree.append(dep_tree2[:9])
                dep_tree2 = dep_tree2[9::]
            elif dep_tree2[8]=='_' and dep_tree2[9]=='_':
                heads_tree.append(dep_tree2[:10])
                dep_tree2 = dep_tree2[10::]
            else:
                heads_tree.append(dep_tree2[:9])
                dep_tree2 = dep_tree2[9::]

    # check where is tree depth and save
    for i in range(len(heads_tree)):
        if heads_tree[i][6].isnumeric():
            heads.append(int(heads_tree[i][6]))
        elif heads_tree[i][5].isnumeric():
            heads.append(int(heads_tree[i][5]))

    # add tree depth
    t_depth.append(max(heads))
    # calculate MDD value
    if (len(heads)-1)==0:
        mdd.append(float("{:.2f}".format(sum(heads))))
    else:
        mdd.append(float("{:.2f}".format(sum(heads)/(len(heads)-1))))

    NN = 0
    VB = 0
    JJ = 0
    PREPOSITION = 0
    DEF = 0
    CONJ = 0
    BN = 0
    NNP = 0
    NNT = 0
    TEMP = 0
    RB = 0
    CC = 0
    AT = 0
    QW = 0
    REL = 0
    PRP = 0
    IN = 0
    DTT = 0
    DT = 0
    MD = 0

    # find and calculate POS in the sentence
    for w in range(len(dep_tree2)):
        if dep_tree2[w] == "NN":
            NN=NN+1; 
        if dep_tree2[w] == "VB":
            VB=VB+1;
        if dep_tree2[w] == "JJ":
            JJ=JJ+1;
        if dep_tree2[w] == "PREPOSITION":
            PREPOSITION=PREPOSITION+1;
        if dep_tree2[w] == "DEF":
            DEF=DEF+1;
        if dep_tree2[w] == "CONJ":
            CONJ=CONJ+1;
        if dep_tree2[w] == "BN":
            BN=BN+1;
        if dep_tree2[w] == "NNP":
            NNP=NNP+1;
        if dep_tree2[w] == "NNT":
            NNT=NNT+1;
        if dep_tree2[w] == "TEMP":
            TEMP=TEMP+3;
        if dep_tree2[w] == "RB":
            RB=RB+1;
        if dep_tree2[w] == "CC":
            CC=CC+1;
        if dep_tree2[w] == "AT":
            AT=AT+1;
        if dep_tree2[w] == "QW":
            QW=QW+1;
        if dep_tree2[w] == "REL":
            REL=REL+1;
        if dep_tree2[w] == "PRP":
            PRP=PRP+1;
        if dep_tree2[w] == "IN":
            IN=IN+1;
        if dep_tree2[w] == "DTT":
            DTT=DTT+1;
        if dep_tree2[w] == "DT":
            DT=DT+1;
        if dep_tree2[w] == "MD":
            MD=MD+1;

    
    NN_l.append(NN)
    VB_l.append(VB)
    JJ_l.append(JJ)
    PREPOSITION_l.append(PREPOSITION)
    DEF_l.append(DEF)
    CONJ_l.append(CONJ)
    BN_l.append(BN)
    NNP_l.append(NNP)
    NNT_l.append(NNT)
    TEMP_l.append(TEMP)
    RB_l.append(RB)
    CC_l.append(CC)
    AT_l.append(AT)
    QW_l.append(QW)
    REL_l.append(REL)
    PRP_l.append(PRP)
    IN_l.append(IN)
    DTT_l.append(DTT)
    DT_l.append(DT)
    MD_l.append(MD)

In [ ]:
df_final = pd.DataFrame({'sentence' : bef.sentence,
                    'mdd' : mdd,
                    't_depth' : t_depth,
                    'no_change' : bef.no_change,
                    'summarizing' : bef.summarizing,
                    'bigrams' : bigrams,
                    'words': words, 
                    'letters': letters, 
                    'word_avg': word_avg,
                    'numbers': numbers, 
                    'symbols': symbols,
                    'if_question': if_question,
                    'punctuation': punctuation,
                    'sym_hashtag': sym_hashtag,
                    'sym_shtrudel': sym_shtrudel,
                    'sym_dollar': sym_dollar,
                    'sym_ampersand': sym_ampersand,
                    'sym_percend': sym_percend,
                    'sym_star': sym_star,
                    'sym_tilda': sym_tilda,
                    'sym_math': sym_math,
                    'sym_logic': sym_logic,
                    'sym_brackets': sym_brackets,
                    'sym_num': sym_num,
                    'sym_quotes': sym_quotes,
                    'sym_slashes': sym_slashes,
                    'NN' : NN_l,
                    'VB' : VB_l,
                    'JJ' : JJ_l,
                    'PREPOSITION' : PREPOSITION_l,
                    'DEF' : DEF_l,
                    'CONJ' : CONJ_l,
                    'BN' : BN_l,
                    'NNP' : NNP_l,
                    'NNT' : NNT_l,
                    'TEMP' : TEMP_l,
                    'RB' : RB_l,
                    'CC' : CC_l,
                    'AT' : AT_l,
                    'QW' : QW_l,
                    'REL' : REL_l,
                    'PRP' : PRP_l,
                    'IN' : IN_l,
                    'DTT' : DTT_l,
                    'DT' : DT_l,
                    'MD' : MD_l})

df_final.head()

In [ ]:
df_final.shape

In [ ]:
#save final file as excel
df_final.to_csv('sentences_521_features_46.csv', index = False, encoding='utf-8-sig')